In [ ]:
import pandas as pd
import csv
import json
import mysql.connector
from mysql.connector import errorcode

from dotenv import load_dotenv
import os

In [ ]:
# Una vez ya tenemos los datos, ahora necesitamos el puente a MySQL 
# Creamos las funciones que necesita nuestro puente

# Recordar que tenemos que tener hecho pip install python-dotenv
# Y en esta misma carpeta en la que ejecutáis, tener un archivo .env 
# que dentro ponga MYSQL_PASSWORD = 'aquí ponéis vuestra contraseña'

load_dotenv()
password = os.getenv("MYSQL_PASSWORD")

# Función conectar
load_dotenv()

def conectar():
    try:
        cnx = mysql.connector.connect(
            user='root',
            password=os.getenv("MYSQL_PASSWORD"),
            host='127.0.0.1',
            auth_plugin='mysql_native_password')
        
        print('Llamada (( ☎ )) a SQL\n')

    except mysql.connector.Error as err:
        print(f'Error: {err}')

    return cnx

# Llama a la función
cnx = conectar()
cursor = cnx.cursor()
print('SQL ha contestado (( ☎ )) está listo para trabajar\n')


Llamada (( ☎ )) a SQL

SQL ha contestado (( ☎ )) está listo para trabajar



In [ ]:
def crear_base_de_datos_y_tablas():
    cnx = conectar()
    if cnx is None:
        return

    cursor = cnx.cursor()
    try:
        # Creamos la base de datos
        cursor.execute("CREATE DATABASE IF NOT EXISTS MusicStreamStupendo")
        cursor.execute("USE MusicStreamStupendo")

        # Tabla artistas (corregido)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas (
            id_artista VARCHAR(45) PRIMARY KEY,
            nombre VARCHAR(100),
            biografia TEXT,
            reproducciones INT,
            oyentes INT,
            artistas_similares TEXT
        )
        """)

        # Tabla generos
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS generos (
            id_genero INT AUTO_INCREMENT PRIMARY KEY,
            nombre VARCHAR(50)
        )
        """)

        # Tabla albumes
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS albumes (
            id_album VARCHAR(50) PRIMARY KEY,
            nombre VARCHAR(100),
            año_lanzamiento DATE,
            popularidad INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Tabla top_tracks
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS top_tracks (
            id_cancion INT PRIMARY KEY,
            titulo VARCHAR(100),
            año_lanzamiento VARCHAR(45),
            id_genero INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Tabla artistas_generos
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas_generos (
            id_artista VARCHAR(45),
            id_genero INT,
            PRIMARY KEY (id_artista, id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero)
        )
        """)

        # Tabla albumes_canciones
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS albumes_canciones (
            id_album VARCHAR(50),
            id_cancion INT,
            PRIMARY KEY (id_album, id_cancion),
            FOREIGN KEY (id_album) REFERENCES albumes(id_album),
            FOREIGN KEY (id_cancion) REFERENCES top_tracks(id_cancion)
        )
        """)

        cnx.commit()
        print('> Base de datos y tablas creadas correctamente.')
    except mysql.connector.Error as err:
        print(f'Error al crear tablas: {err}')

    finally:
        cursor.close()
        cnx.close()

# Llamar a la función para crear la base de datos y tablas
crear_base_de_datos_y_tablas()


Llamada (( ☎ )) a SQL


 > Base de datos y tablas creadas correctamente.


In [ ]:
# Empezamos a empujar, primero subimos artista_id porque es una Primary Key
# y está en otro CSV, ésa era la piedra

# van a aparecer duplicados descartados al cargar, ni caso, MySQL los dejará en tierra

artista_id = pd.read_csv('../csv/total_canciones_2018-2022.csv')

for columna in artista_id.itertuples(index=False):
    tipo, artista, artista_id, genero, nombre, año, id, popularidad_cancion = columna

    try:
        cursor.execute("USE MusicStreamStupendo")
        sql = """INSERT INTO artistas (id_artista, nombre)
                VALUES (%s, %s)"""
        cursor.execute(sql, (artista_id, artista))
        print('Tout va bien: sin incidencias.')
    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
    else:
        cnx.commit()

Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1062 (23000): Duplicate entry '77kULmXAQ6vWer7IIHdGzI' for key 'artistas.PRIMARY'
Rien de rien: 1062 (23000): Duplicate entry '77kULmXAQ6vWer7IIHdGzI' for key 'artistas.PRIMARY'
Tout va bien: sin incidencias.
Rien de rien: 1062 (23000): Duplicate entry '13ZEDW6vyBF12HYcZRr4EV' for key 'artistas.PRIMARY'
Tout va bien: sin incidencias.
Rien de rien: 1062 (23000): Duplicate entry '70kkdajctXSbqSMJbQO424' for key 'artistas.PRIMARY'
Rien de rien: 1062 (23000): Duplicate entry '70kkdajctXSbqSMJbQO424' for key 'artistas.PRIMARY'
Rien de rien: 1062 (23000): Duplicate entry '70kkdajctXSbqSMJbQO424' for key 'artistas.PRIMARY'
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1062 (23000): Duplicate entry '718COspgdWOnwOFpJHRZHS' for key 'artistas.PRIMARY'
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1062

In [ ]:
# Seguimos empujando
# Usamos UPDATE para que nos encajen los campos y no se exporte de forma loca

artistas = pd.read_csv('../csv/datos_artistas_lastfm.csv')

for columna in artistas.itertuples(index=False):
    artista, playcount, listeners, biografia_resumen, similares = columna
    try:
        cursor.execute("USE MusicStreamStupendo")
        sql = """UPDATE artistas
                SET reproducciones = %s,
                    oyentes = %s,
                    biografia = %s,
                    artistas_similares = %s
                WHERE nombre = %s"""
        
        cursor.execute(sql, (playcount, listeners, biografia_resumen, similares, artista))

        print(f'Tout va bien: {artista} actualizado.')

    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
        
    else:
        cnx.commit()



Tout va bien: Ozzy Osbourne actualizado.
Tout va bien: Omar Apollo actualizado.
Tout va bien: Airbag actualizado.
Tout va bien: Aitana actualizado.
Tout va bien: Masego actualizado.
Tout va bien: Dayseeker actualizado.
Tout va bien: Nevertel actualizado.
Tout va bien: Germ actualizado.
Tout va bien: Kris Kristofferson actualizado.
Tout va bien: OneRepublic actualizado.
Tout va bien: ZillaKami actualizado.
Tout va bien: Cage The Elephant actualizado.
Tout va bien: In Flames actualizado.
Tout va bien: Eluveitie actualizado.
Tout va bien: Bastille actualizado.
Tout va bien: Colter Wall actualizado.
Tout va bien: AJR actualizado.
Rien de rien: 1054 (42S22): Unknown column 'nan' in 'field list'
Tout va bien: Tom Odell actualizado.
Tout va bien: Janelle Monáe actualizado.
Tout va bien: Luke Combs actualizado.
Rien de rien: 1054 (42S22): Unknown column 'nan' in 'field list'
Tout va bien: Chase Atlantic actualizado.
Tout va bien: Cody Jinks actualizado.
Tout va bien: Killswitch Engage actualiz

In [ ]:
# Hay unos artistas que no han obtenido resultados en last.fm, nos los quitamos de encima
# ya que no podremos consultarlos

try:
    cursor.execute("USE MusicStreamStupendo")
    sql = """DELETE FROM artistas
            WHERE reproducciones IS NULL
               OR oyentes IS NULL
               OR biografia IS NULL
               OR artistas_similares IS NULL"""
    cursor.execute(sql)
    print(f'Artistas con datos NULL eliminados: {cursor.rowcount}')
except Exception as e:
    cnx.rollback()
    print('Error al eliminar artistas:', e)
else:
    cnx.commit()

Artistas con datos NULL eliminados: 42
